# Compare North Atlantic chlorophyll values to krill relative abundance from CPR surveys

In [2]:

import os
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
import seaborn as sb
sb.set(style='ticks')
import cartopy.crs as ccrs
import cartopy.feature as cfeature

os.getcwd()


'/home/users/train013/CMIP6_hackathon'

In [3]:
#%% get data

### krill data
os.chdir('/gws/pw/j05/cop26_hackathons/bristol/project09/krill')
data = nc.Dataset('CPR_bin.nc')
krill = data.variables['count'][...]
effort = data.variables['effort'][...]
krill_lon = data.variables['longitude'][...]
krill_lat = data.variables['latitude'][...]
krill_year = data.variables['time'][...]

### wrap the longitidues to agree with the chlorophyll data
krill_lon = np.ma.concatenate((krill_lon[:,200::], krill_lon[:,0:200]+360.0), axis=1)
krill = np.ma.concatenate((krill[:,:,200::], krill[:,:,0:200]), axis=2)
effort = np.ma.concatenate((effort[:,:,200::], effort[:,:,0:200]), axis=2)


chl = np.zeros((12,180,360))

### Ocean chlorophyll data
os.chdir('/gws/pw/j05/cop26_hackathons/bristol/project09/chlorophyll')
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-01-fv4.2.nc','r')
chl[0,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-02-fv4.2.nc','r')
chl[1,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-03-fv4.2.nc','r')
chl[2,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-04-fv4.2.nc','r')
chl[3,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-05-fv4.2.nc','r')
chl[4,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-06-fv4.2.nc','r')
chl[5,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-07-fv4.2.nc','r')
chl[6,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-08-fv4.2.nc','r')
chl[7,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-09-fv4.2.nc','r')
chl[8,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-10-fv4.2.nc','r')
chl[9,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-11-fv4.2.nc','r')
chl[10,:,:] = data.variables['chlor_a'][...]
data = nc.Dataset('ETOPO_ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-12-fv4.2.nc','r')
chl[11,:,:] = data.variables['chlor_a'][...]

lon = data.variables['ETOPO60X'][...]
lat = data.variables['ETOPO60Y'][...]

data.close()


### mask bad values
chl = np.ma.masked_where(chl > 1e5, chl)




KeyError: 'effort'

In [ ]:
#%% create average values over the year for chlorophyll and sum of all krill counts across years

chl_ave = np.ma.average(chl, axis=0)

### counts / effort and averaged over all years
krill_abu = krill / effort
krill_abu = np.ma.average(krill_abu, axis=0)

### remove krill relative abundance where it equals 1 and select only north atlantic region
krill_abu = np.ma.masked_where(krill_abu == 1.0, krill_abu)
llon, llat = np.meshgrid(lon,lat)
krill_abu = np.ma.masked_where(llon < 270.0, krill_abu)
krill_abu = np.ma.masked_where(llat < 35, krill_abu)
krill_abu = np.ma.masked_where(llat > 70, krill_abu)

### mask chlorophyll where krill are not
mask = np.ma.getmask(krill_abu)
chl_m = np.ma.masked_where(mask, chl_ave)
krill_abu = np.ma.masked_where(np.ma.getmask(chl_m), krill_abu)


In [ ]:
#%% calculate density of points 

from scipy.stats import gaussian_kde

def get_gaussian(x1,y1):

    x = x1
    y = y1

    x = x.compressed()
    y = y.compressed()

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    return x,y,z


[chlx,chly,chlz] = get_gaussian(chl_m, krill_abu)

### deal with zeros
tmp = chlx*chly
chlx = np.ma.compressed(np.ma.masked_where(tmp == 0.0, chlx))
chly = np.ma.compressed(np.ma.masked_where(tmp == 0.0, chly))
chlz = np.ma.compressed(np.ma.masked_where(tmp == 0.0, chlz))

chl_log10 = np.log10(chlx)
kri_log10 = np.log10(chly)

In [ ]:
#%% have a look

proj = ccrs.Robinson(central_longitude=20)

levs1 = np.arange(0,101,5)*0.1
levs2 = np.arange(-100,1,5)*0.01

colmap1 = cm.viridis
colmap2 = cm.viridis

fstic = 13
fslab = 15

fig = plt.figure(figsize=(6.5,9))
gs = GridSpec(3,10)

ax1 = plt.subplot(gs[0,0:8], projection=proj)
ax1.set_extent([-90,20,30,70])
p1 = plt.contourf(krill_lon, krill_lat, krill_abu, transform=ccrs.PlateCarree(), cmap=colmap1, levels=levs1, vmin=np.min(levs1), vmax=np.max(levs1), extend='both')
#c1 = plt.contour(lon, lat, s2n_npp_jan[0,:,:], transform=ccrs.PlateCarree(), colors='k', linewidths=0.75, levels=[-1,1])
ax1.add_feature(cfeature.LAND, color='silver', zorder=2)
ax1.coastlines(zorder=2)

ax2 = plt.subplot(gs[1,0:8], projection=proj)
ax2.set_extent([-90,20,30,70])
p2 = plt.contourf(lon, lat, np.log10(chl_ave[:,:]), transform=ccrs.PlateCarree(), cmap=colmap2, levels=levs2, vmin=np.min(levs2), vmax=np.max(levs2), extend='both')
c2 = plt.contour(lon, lat, np.log10(chl_ave[:,:]), transform=ccrs.PlateCarree(), colors='k', linewidths=0.75, levels=levs2[::2])
ax2.add_feature(cfeature.LAND, color='silver', zorder=2)
ax2.coastlines(zorder=2)

ax3 = plt.subplot(gs[2,1:])
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.tick_params(labelsize=fstic)
plt.scatter(chl_log10, kri_log10, c=chlz, cmap='copper_r', s=20, alpha=0.5)

'''
from scipy.optimize import curve_fit
def linreg(x,a,b):
    return a*x + b
coef, cova = curve_fit(linreg, chl_log10, kri_log10, method='lm')
print(coefs)
xx = np.linspace(np.min(chl_log10), np.max(chl_log10), 100)
yy = linreg(xx, coef[0], coef[1])
plt.plot(xx, yy, color='firebrick', linewidth=1.5, linestyle='-', alpha=0.75)
'''

plt.plot((-1,-1),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((-0.8,-0.8),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((-0.6,-0.6),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((-0.4,-0.4),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((-0.2,-0.2),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((0,0),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((0.2,0.2),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((0.4,0.4),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((0.6,0.6),(-1,3), 'k--', alpha=0.5, linewidth=0.5)
plt.plot((0.8,0.8),(-1,3), 'k--', alpha=0.5, linewidth=0.5)

plt.xlim(-1,1)
plt.ylim(-1,2)

plt.ylabel('log$_{10}$(counts per effort)', fontsize=fslab)
plt.xlabel('log$_{10}$(chlorophyll-a) (mg m$^{-3}$)', fontsize=fslab)

x = 0.05; y = 1.05
plt.text(x,y,'a', transform=ax1.transAxes, fontweight='bold', fontsize=fslab+2, ha='center', va='center')
plt.text(x,y,'b', transform=ax2.transAxes, fontweight='bold', fontsize=fslab+2, ha='center', va='center')
plt.text(x,y,'c', transform=ax3.transAxes, fontweight='bold', fontsize=fslab+2, ha='center', va='center')

plt.subplots_adjust(top=0.95)

cbax1 = fig.add_axes([0.8, 0.725, 0.05, 0.2])
cbar1 = plt.colorbar(p1, cax=cbax1, orientation='vertical', ticks=levs1[::2])
cbar1.ax.set_ylabel('counts per effort', fontsize=fslab)
cbar1.ax.tick_params(labelsize=fstic)

cbax2 = fig.add_axes([0.8, 0.43, 0.05, 0.2])
cbar2 = plt.colorbar(p2, cax=cbax2, orientation='vertical', ticks=levs2[::2])
cbar2.ax.set_ylabel('log$_{10}$(mg m$^{-3}$)', fontsize=fslab)
cbar2.ax.tick_params(labelsize=fstic)

fig.savefig('Chlorophyll_krill.png', dpi=300, bbox_inches='tight')
